<a href="https://colab.research.google.com/github/sergiolucero/notebooks/blob/main/PadronS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Revisando el padrón
Will this cost me in s3 xfer fees?

In [2]:
import pandas as pd  # try dask?
pdf = pd.read_csv('s3://quantcldata/CLIENTES/SERVEL/padron_servel_2020.csv.gz')

In [3]:
pdf.head()

,nombre,rut,genero,direccion,region,comuna
0,A BERNAD CONSUELO TERESITA,14.520.314-7,MUJER,LOS JACINTOS 1960,1,IQUIQUE
1,AABY ARESTIZABAL JANETH PAMELA,8.896.601-5,MUJER,AV LA TIRANA 3549,1,IQUIQUE
2,AATWANI HEMLATA NIHALCHAND,21.920.429-9,MUJER,FRANCISCO BILBAO 3421 D 1604,1,IQUIQUE
3,ABAETY ARAPUCA RITA,24.769.580-K,MUJER,CESPEDES Y GONZALEZ 1802,1,IQUIQUE
4,ABALLAI ASTARGO MIGUEL ANGEL,9.402.572-9,VARON,MANUEL CASTRO RAMOS 2311,1,IQUIQUE


In [4]:
itu = pdf[pdf.nombre.str.contains('ITURRIAGA')]
len(itu)

5976

In [5]:
itu.comuna.value_counts()

CURICO          477
TENO            293
SAN BERNARDO    201
MAIPU           192
BUIN            186
               ... 
QUILLECO          1
MULCHEN           1
LAJA              1
LOS ALAMOS        1
SAN NICOLAS       1
Name: comuna, Length: 244, dtype: int64

In [7]:
itu[itu.nombre.str.contains('ORTUZAR')]

,nombre,rut,genero,direccion,region,comuna
9836086,ITURRIAGA ORTUZAR GONZALO HERNAN,6.698.826-0,VARON,MEDINACELLI 1121,13,LAS CONDES
9836087,ITURRIAGA ORTUZAR JUAN PABLO,7.014.263-5,VARON,MEDINACELLI 1121,13,LAS CONDES
12782972,ITURRIAGA ORTUZAR SEBASTIAN,13.436.311-8,VARON,AVENIDA CHAMISERO 13178 12 CHAMISERO,13,COLINA


In [13]:
# splitsville
for regcom, rcdf in pdf.groupby(['region','comuna']):
  #print(regcom, len(rcdf))
  reg, com = regcom
  reg = '%02d' %reg
  rfn = f'R{reg}_C{com}.pq'
  rcdf.to_parquet(rfn, index=False)
  if 'SAN' in com:
    print(regcom, len(rcdf))

(2, 'SAN PEDRO DE ATACAMA') 6718
(5, 'SAN ANTONIO') 78146
(5, 'SAN ESTEBAN') 14795
(5, 'SAN FELIPE') 59722
(5, 'SANTA MARIA') 11856
(5, 'SANTO DOMINGO') 10784
(6, 'SAN FERNANDO') 62452
(6, 'SAN VICENTE') 40151
(6, 'SANTA CRUZ') 30911
(7, 'SAN CLEMENTE') 36534
(7, 'SAN JAVIER') 38701
(7, 'SAN RAFAEL') 8242
(8, 'SAN PEDRO DE LA PAZ') 86974
(8, 'SAN ROSENDO') 4386
(8, 'SANTA BARBARA') 13158
(8, 'SANTA JUANA') 12537
(10, 'SAN JUAN DE LA COSTA') 7837
(10, 'SAN PABLO') 9820
(12, 'SAN GREGORIO') 886
(13, 'SAN BERNARDO') 226795
(13, 'SAN JOAQUIN') 82967
(13, 'SAN JOSE DE MAIPO') 15474
(13, 'SAN MIGUEL') 103746
(13, 'SAN PEDRO') 8184
(13, 'SAN RAMON') 82197
(13, 'SANTIAGO') 331401
(16, 'SAN CARLOS') 47270
(16, 'SAN FABIAN') 4758
(16, 'SAN IGNACIO') 14354
(16, 'SAN NICOLAS') 11370


In [10]:
pdf.region.value_counts()

13    5707250
5     1575539
8     1319625
7      871133
9      869370
6      758105
10     726202
4      601442
2      470418
16     421523
14     344870
1      254078
3      238864
15     189798
12     158627
11      96577
Name: region, dtype: int64

In [14]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14603421 entries, 0 to 14603420
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   nombre     object
 1   rut        object
 2   genero     object
 3   direccion  object
 4   region     int64 
 5   comuna     object
dtypes: int64(1), object(5)
memory usage: 668.5+ MB


In [ ]:
!du -hs *pq

In [17]:
import dask.dataframe as dd
df = dd.read_parquet('*.pq')
df

,nombre,rut,genero,direccion,region,comuna
npartitions=345,,,,,,
,object,object,object,object,int64,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [18]:
import time
t0=time.time()
pad = df[df.nombre.str.contains('ITURRIAGA')].compute()
print(len(pad), time.time()-t0)

5976 44.31248712539673


In [21]:
t0=time.time()
padLu = pdf[pdf.nombre.str.contains('YAÑEZ')]
print(len(padLu), time.time()-t0)

78920 9.852583885192871
